In [16]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
df=pd.read_csv('data.csv',encoding='unicode_escape')
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

1) Preprocess the data. How are you handling nulls? What process(es) are you using to encode and normalize the data?

In [17]:

df.dropna(inplace=True)
#I handled the nulls by removing the rows with missing data
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')
df["Description"] = df["Description"].str.lower()


df = df[df['Quantity'] > 0]

df.insert(loc=2, column='year_month', value=df['InvoiceDate'].map(lambda x: 100*x.year + x.month))

df.insert(loc=3, column='month', value=df.InvoiceDate.dt.month)


df.insert(loc=4, column='day', value=(df.InvoiceDate.dt.dayofweek)+1)

df.insert(loc=5, column='hour', value=df.InvoiceDate.dt.hour)

df["amount_spent"] = df["UnitPrice"]*df["Quantity"]

df["StockCode"] = df.StockCode.astype(float)


#Here I made the data easier to read by changing formatting and also adding a column to help understand the 
#data better.



ValueError: could not convert string to float: '85123A'

Perform k-means on this dataset for customer segments. Customer segments help determine what types of people buy your product, which allows you to target more people like your usual customers. Should you look at all the data, or which subset of data should you use? What is the ideal number of clusters? Which approach did you use to find the ideal number of clusters and why?

In [ ]:
inertia = []
customer_features = ['amount_spent', 'Quantity', 'InvoiceNo']
X = df[customer_features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_scaled)
df['clusters'] = kmeans.labels_

def elbow_method(X, k):
  kmeans = KMeans(n_clusters=k, random_state=0)
  kmeans.fit(X)
  return kmeans.inertia_

print(kmeans.cluster_centers_)



[[ 7.27425619e-04 -5.62369214e-04 -9.59928532e-01]
 [ 5.45039220e+02  4.48852598e+02  1.59206719e+00]
 [ 2.49668037e+02  4.11273612e+02 -1.46390243e+00]
 [-4.24814054e-03 -3.48275916e-03  7.93077706e-01]]


You should not use the entire dataset, but the sections that show frequency and amount spent. This way we know how much each customer bought and how much they spent, which allowed us to figure out what is more popular. I used the elbow method as it



 Perform hierarchical clustering for customer segments. What is the ideal number of clusters? Which approach did you use to find the ideal number of clusters and why?

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage
# Perform hierarchical clustering
clustering = AgglomerativeClustering(n_clusters=5)  # Choose the number of clusters
clustering.fit(X)

linked = linkage(X, method='ward')  # Choose linkage method
df['Cluster'] = clustering.labels_
cluster_means = df.groupby('Cluster').mean()
print(cluster_means)


 Compare the results of 2 and 3. Which approach do you think is best? Why?